In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train = pd.read_csv('/kaggle/input/titanic/train.csv')
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [217]:
test = pd.read_csv('/kaggle/input/titanic/test.csv')
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [5]:
display(train.shape)
display(test.shape)

(891, 12)

(418, 11)

In [6]:
display(train.dtypes)

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [7]:
display(train['Pclass'].unique())
display(train.groupby(['Pclass'])[['Survived']].apply(lambda x: x.count()))

array([3, 1, 2])

,Survived
Pclass,
1,216
2,184
3,491


In [8]:
display(train.groupby(['Sex'])[['Survived']].apply(lambda x: x.count()))

,Survived
Sex,
female,314
male,577


In [9]:
display(train['Age'].describe())
# Define age bins
bins = [0, 18, 30, 40, 50, 60, 70, 80]
labels = ['0-18', '19-30', '31-40', '41-50', '51-60', '61-70', '71-80']

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

In [10]:
train['Age Group'] = pd.cut(train['Age'], bins=bins, labels=labels, right=False)
display(train.groupby(['Age Group'])[['Survived']].apply(lambda x: x.count()))

/tmp/ipykernel_30/3051666627.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  display(train.groupby(['Age Group'])[['Survived']].apply(lambda x: x.count()))


,Survived
Age Group,
0-18,113
19-30,271
31-40,167
41-50,89
51-60,48
61-70,19
71-80,6


In [11]:

display(train['Embarked'].unique())
embarked_mapping = {
    'nan': 0,
     np.nan: 0,
    'S': 1,
    'C': 2,
    'Q': 3
}
train_plne = train.copy()
test_plne = test.copy()
train_plne['Embarked'] = train_plne['Embarked'].replace(embarked_mapping)
test_plne['Embarked'] = test_plne['Embarked'].replace(embarked_mapping)
train_plne['Sex'] = train_plne['Sex'].replace({'male': 1, 'female': 0})
test_plne['Sex'] = test_plne['Sex'].replace({'male': 1, 'female': 0})
train_median_age = train_plne['Age'].median()
test_median_age = test_plne['Age'].median()

# Impute NaN values in the Age column with the median age
train_plne['Age'].fillna(train_median_age, inplace=True)
test_plne['Age'].fillna(test_median_age, inplace=True)
display(train_plne.head())
display(test_plne.head())

array(['S', 'C', 'Q', nan], dtype=object)

/tmp/ipykernel_30/1576044785.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_plne['Embarked'] = train_plne['Embarked'].replace(embarked_mapping)
/tmp/ipykernel_30/1576044785.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_plne['Embarked'] = test_plne['Embarked'].replace(embarked_mapping)
/tmp/ipykernel_30/1576044785.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age Group
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,1,19-30
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,2,31-40
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,1,19-30
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,1,31-40
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,1,31-40


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",1,34.5,0,0,330911,7.8292,NaN,3
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",0,47.0,1,0,363272,7.0000,NaN,1
2,894,2,"Myles, Mr. Thomas Francis",1,62.0,0,0,240276,9.6875,NaN,3
3,895,3,"Wirz, Mr. Albert",1,27.0,0,0,315154,8.6625,NaN,1
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,22.0,1,1,3101298,12.2875,NaN,1


In [12]:
nan_counts = train_plne.isna().sum()
display(nan_counts)
nan_counts = test_plne.isna().sum()
display(nan_counts)

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
Age Group      178
dtype: int64

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [13]:
train_plne.dtypes

PassengerId       int64
Survived          int64
Pclass            int64
Name             object
Sex               int64
Age             float64
SibSp             int64
Parch             int64
Ticket           object
Fare            float64
Cabin            object
Embarked          int64
Age Group      category
dtype: object

In [14]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [15]:
train_num = train_plne[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
test_num = test_plne[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
train_num_standardized = (train_num - train_num.min()) / (train_num.max() - train_num.min())
test_num_standardized = (test_num - test_num.min()) / (test_num.max() - test_num.min())
train_num_standardized['Survived'] = train_plne['Survived']
X = train_num_standardized[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']].values
y = train_num_standardized['Survived'].values
# Convert to tensors
features = torch.tensor(X, dtype=torch.float32)
predicted = torch.tensor(y, dtype=torch.float32)
display(features)


X = test_num_standardized[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']].values
test_data = torch.tensor(X, dtype=torch.float32)

tensor([[1.0000, 1.0000, 0.2712,  ..., 0.0000, 0.0142, 0.3333],
        [0.0000, 0.0000, 0.4722,  ..., 0.0000, 0.1391, 0.6667],
        [1.0000, 0.0000, 0.3214,  ..., 0.0000, 0.0155, 0.3333],
        ...,
        [1.0000, 0.0000, 0.3466,  ..., 0.3333, 0.0458, 0.3333],
        [0.0000, 1.0000, 0.3214,  ..., 0.0000, 0.0586, 0.6667],
        [1.0000, 1.0000, 0.3968,  ..., 0.0000, 0.0151, 1.0000]])

In [16]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device")

Using cuda device


In [17]:
from torch.utils.data import Dataset, DataLoader, random_split
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [18]:
# Create an instance of the dataset
dataset = CustomDataset(features, predicted)
# Define the split sizes (e.g., 80% train, 20% validation)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [175]:
batch_size = 1

# Create data loaders.
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [176]:
x = train_dataloader.dataset[0][0]
x

tensor([0.5000, 1.0000, 0.7361, 0.0000, 0.0000, 0.0264, 0.3333])

In [177]:
lin1 = torch.nn.Linear(7, 14)
activation = torch.nn.ReLU()
lin2 = torch.nn.Linear(14, 2)
softmax = torch.nn.Softmax()

x = lin1(x)
x = activation(x)
x = lin2(x)
y = softmax(x)
y

tensor([0.5365, 0.4635], grad_fn=<SoftmaxBackward0>)

In [178]:
class TinyModel(nn.Module):
    def __init__(self):
        super(TinyModel, self).__init__()
        
        self.linear1 = nn.Linear(7, 14)
        self.activation = nn.ReLU()
        self.linear2 = nn.Linear(14, 7)
        self.activation = nn.ReLU()
        self.linear3 = nn.Linear(7, 1)
        
    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        x = self.activation(x)
        x = self.linear3(x)
        return x
    
model = TinyModel()     
model = model.to(device)    

In [179]:
print('\n\nModel params:')
for param in model.parameters():
    print(param)



Model params:
Parameter containing:
tensor([[-0.2589, -0.2461, -0.1109, -0.1349, -0.2661, -0.1105, -0.2132],
        [ 0.0772, -0.3746, -0.1433, -0.3405,  0.1439,  0.1425,  0.2389],
        [ 0.3346,  0.3729,  0.1002,  0.1692,  0.1972, -0.1535, -0.3019],
        [-0.1932,  0.0133,  0.0703, -0.2873, -0.2090,  0.2768,  0.2898],
        [-0.0688,  0.2173,  0.1144, -0.0868, -0.2752,  0.1032,  0.2049],
        [-0.1788,  0.3034, -0.2655,  0.0081, -0.3168, -0.2152,  0.1201],
        [ 0.3496, -0.3420,  0.2720,  0.3247, -0.2572, -0.0661, -0.0341],
        [ 0.2547,  0.2666,  0.2011,  0.3129,  0.0811,  0.2421, -0.3063],
        [-0.0739, -0.0802,  0.3477,  0.0611, -0.3205,  0.1092, -0.1991],
        [-0.3040,  0.0252,  0.0076,  0.2578, -0.3720,  0.0179,  0.1428],
        [ 0.1548,  0.0449, -0.2326, -0.2004,  0.2293,  0.1016,  0.2242],
        [-0.2422, -0.0151, -0.1314,  0.0991,  0.2138,  0.2971, -0.2851],
        [-0.0765, -0.0009,  0.1263,  0.3325, -0.1648, -0.3642,  0.3307],
        [-0.2

In [198]:
loss_fn = nn.BCEWithLogitsLoss()  # Binary Cross Entropy Loss for binary classification
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [199]:
def train(dataloader, model, loss_fn, optimizer):
    model.train()
    for X,y in dataloader:
        optimizer.zero_grad()
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y.unsqueeze(-1))

        # Backpropagation
        loss.backward()
        optimizer.step()

In [200]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            labels = model(X)
            test_loss += loss_fn(labels, y.unsqueeze(-1)).item()
            predictions = (labels > 0.5).float()
            correct += (predictions == y).sum().item()
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.2f}%, Avg loss: {test_loss:>8f} \n")

In [201]:
per = 0.4
((per > 0.5) == y)

tensor([False, False])

In [202]:
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(val_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
Test Error: 
 Accuracy: 76.54%, Avg loss: 83.936748 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 76.54%, Avg loss: 83.418171 

Epoch 3
-------------------------------
Test Error: 
 Accuracy: 76.54%, Avg loss: 83.032246 

Epoch 4
-------------------------------
Test Error: 
 Accuracy: 76.54%, Avg loss: 82.671546 

Epoch 5
-------------------------------
Test Error: 
 Accuracy: 77.09%, Avg loss: 82.515991 

Epoch 6
-------------------------------
Test Error: 
 Accuracy: 76.54%, Avg loss: 82.803885 

Epoch 7
-------------------------------
Test Error: 
 Accuracy: 77.65%, Avg loss: 82.380386 

Epoch 8
-------------------------------
Test Error: 
 Accuracy: 77.65%, Avg loss: 82.325383 

Epoch 9
-------------------------------
Test Error: 
 Accuracy: 78.21%, Avg loss: 82.141980 

Epoch 10
-------------------------------
Test Error: 
 Accuracy: 78.21%, Avg loss: 82.092749 

Epoch 11
-------------------------------
Test Error: 
 Accu

In [223]:
ids = []
predictions = []
for (X,id) in zip(test_dataloader, test['PassengerId'].values):
    label = model(X.to(device))
    ids.append(id)
    predictions.append((int((label > 0.5).float().sum().item())))

In [228]:
# Create a DataFrame from the lists
df = pd.DataFrame({
    'PassengerId': ids,
    'Survived': predictions
})

# Save the DataFrame to a CSV file without the default index
df.to_csv('submission.csv', index=False)